In [1]:
!pip install pillow lxml

In [2]:
import os
from lxml import etree
from PIL import Image

In [ ]:
# === Настройки ===
input_dir = 'svg_dir'  # Папка с SVG-файлами и изображениями
image_ext = '.png'     
class_map = {'door': 0, 'window': 1}  

In [ ]:
def extract_points_from_polygon(points_str):
    return [tuple(map(float, p.strip().split(','))) 
            for p in points_str.strip().split()]

In [ ]:
def polygon_to_bbox(points):
    x_coords, y_coords = zip(*points)
    return min(x_coords), min(y_coords), max(x_coords), max(y_coords)

In [ ]:
def normalize_bbox(x_min, y_min, x_max, y_max, img_w, img_h):
    x_center = ((x_min + x_max) / 2) / img_w
    y_center = ((y_min + y_max) / 2) / img_h
    width = (x_max - x_min) / img_w
    height = (y_max - y_min) / img_h
    return x_center, y_center, width, height

In [ ]:
def convert_svg_to_yolo(svg_path, image_path, output_txt_path):
    tree = etree.parse(svg_path)
    root = tree.getroot()

    img = Image.open(image_path)
    img_w, img_h = img.size

    lines = []

    for elem in root.iter():
        tag = etree.QName(elem).localname
        if tag != 'polygon':
            continue

        attribs = elem.attrib
        label = attribs.get('class', '') or attribs.get('id', '')
        label = label.lower()

        for name, class_id in class_map.items():
            if name in label:
                points_str = attribs.get('points', '')
                if not points_str:
                    continue
                points = extract_points_from_polygon(points_str)
                x_min, y_min, x_max, y_max = polygon_to_bbox(points)
                x_c, y_c, w, h = normalize_bbox(x_min, y_min, x_max, y_max, img_w, img_h)
                lines.append(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")
                break

    if lines:
        with open(output_txt_path, 'w') as f:
            f.write('\n'.join(lines))
        print(f"Saved: {output_txt_path}")


In [ ]:
# === Основной цикл ===
for filename in os.listdir(input_dir):
    if not filename.endswith('.svg'):
        continue

    base = os.path.splitext(filename)[0]
    svg_path = os.path.join(input_dir, filename)
    image_path = os.path.join(input_dir, base + image_ext)
    output_txt_path = os.path.join(input_dir, base + '.txt')

    if not os.path.exists(image_path):
        print(f"[!] Image not found for {filename}")
        continue

    convert_svg_to_yolo(svg_path, image_path, output_txt_path)